In [1]:
import numpy as np
import pandas as pd
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import accuracy_score

In [2]:
train_data = pd.read_csv("training_car_x_y_train.csv", delimiter = ",")
test_data = pd.read_csv("test_car_x_test.csv", delimiter = ",")

In [3]:
print(train_data.shape)
print(test_data.shape)

(297369, 20)
(74343, 19)


In [4]:
#Understanding data

In [5]:
train_data.head()

,dateCrawled,name,seller,offerType,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen,price
0,2016-03-06 09:45:45,Volkswagen_Golf_Variant_1.9_TDI_Trendline,privat,Angebot,control,kombi,2009,manuell,105,golf,150000,4,diesel,volkswagen,nein,2016-03-06 00:00:00,0,24220,2016-03-26 14:16:37,5999
1,2016-03-25 16:57:48,Twingo_Baujahr_2000,privat,Angebot,control,kleinwagen,2000,manuell,58,twingo,150000,2,benzin,renault,nein,2016-03-25 00:00:00,0,47167,2016-03-28 17:16:47,0
2,2016-03-24 08:36:58,BMW_325d_Touring,privat,Angebot,control,kombi,2007,manuell,197,3er,150000,5,diesel,bmw,ja,2016-03-24 00:00:00,0,74535,2016-03-24 09:50:27,6100
3,2016-04-05 07:36:37,BMW_E36_320i_Touring_M_Paket_Avusblau_individual,privat,Angebot,test,NaN,2000,manuell,150,NaN,150000,9,NaN,bmw,NaN,2016-04-05 00:00:00,0,46047,2016-04-05 09:44:06,1200
4,2016-03-09 17:58:02,Mercedes_Benz_A_140,privat,Angebot,test,limousine,1999,automatik,82,a_klasse,150000,12,benzin,mercedes_benz,nein,2016-03-09 00:00:00,0,25451,2016-03-22 04:44:36,2500


In [6]:
del train_data['dateCrawled']
del train_data['name']
del train_data['model']
del train_data['lastSeen']

del test_data['dateCrawled']
del test_data['name']
del test_data['model']
del test_data['lastSeen']

In [7]:
train_data.head()

,seller,offerType,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,price
0,privat,Angebot,control,kombi,2009,manuell,105,150000,4,diesel,volkswagen,nein,2016-03-06 00:00:00,0,24220,5999
1,privat,Angebot,control,kleinwagen,2000,manuell,58,150000,2,benzin,renault,nein,2016-03-25 00:00:00,0,47167,0
2,privat,Angebot,control,kombi,2007,manuell,197,150000,5,diesel,bmw,ja,2016-03-24 00:00:00,0,74535,6100
3,privat,Angebot,test,NaN,2000,manuell,150,150000,9,NaN,bmw,NaN,2016-04-05 00:00:00,0,46047,1200
4,privat,Angebot,test,limousine,1999,automatik,82,150000,12,benzin,mercedes_benz,nein,2016-03-09 00:00:00,0,25451,2500


In [8]:
set(train_data['seller'])

{'gewerblich', 'privat'}

In [9]:
len(train_data[train_data.seller == 'gewerblich'])

2

## So, we can conviniently remove the seller column

In [10]:
del train_data['seller']
del test_data['seller']

In [11]:
set(train_data['offerType'])

{'Angebot', 'Gesuch'}

In [12]:
len(train_data[train_data.offerType == 'Gesuch'])

11

In [13]:
len(train_data[train_data.offerType == 'Angebot'])

297358

## So, we can also remove the offerType column

In [14]:
del train_data['offerType']
del test_data['offerType']

In [15]:
set(train_data['abtest'])

{'control', 'test'}

In [16]:
len(train_data[train_data.abtest == 'control'])

143318

In [17]:
len(train_data[train_data.abtest == 'test'])

154051

In [18]:
# We will keep 'abtest'

In [19]:
set(train_data['vehicleType'])

{'andere',
 'bus',
 'cabrio',
 'coupe',
 'kleinwagen',
 'kombi',
 'limousine',
 nan,
 'suv'}

In [20]:
len(train_data[train_data.vehicleType == 'kleinwagen'])

64218

In [21]:
len(train_data[train_data.vehicleType == 'andere'])

2698

In [22]:
# We will keep 'vehicleType'

In [23]:
len(train_data[train_data.yearOfRegistration > 2017])

3333

In [24]:
train_data = train_data[(train_data.yearOfRegistration >=1970) & (train_data.yearOfRegistration <= 2017) & (train_data.price >100) & (train_data.price < 200000)]
train_data.shape

(281387, 14)

In [25]:
set(train_data['gearbox'])

{'automatik', 'manuell', nan}

In [26]:
len(train_data[train_data.gearbox == 'automatik'])

59842

In [27]:
len(train_data[train_data.gearbox == 'manuell'])

209057

In [28]:
# We will kepp 'gearbox' column

In [29]:
del train_data['monthOfRegistration']
del test_data['monthOfRegistration']

In [30]:
set(train_data['fuelType'])

{'andere', 'benzin', 'cng', 'diesel', 'elektro', 'hybrid', 'lpg', nan}

In [31]:
# We will keep 'fuelType' column

In [32]:
#We will keep 'brand' column

In [33]:
#We will keep 'notRepairedDamage' column

In [34]:
# we will deduce info from 'dateCreated' column

In [35]:
len(train_data[train_data.nrOfPictures == 0])

281387

## We can remove nrOfPictures as well

In [36]:
del train_data['nrOfPictures']
del test_data['nrOfPictures']

In [37]:
train_data.head()

,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,kilometer,fuelType,brand,notRepairedDamage,dateCreated,postalCode,price
0,control,kombi,2009,manuell,105,150000,diesel,volkswagen,nein,2016-03-06 00:00:00,24220,5999
2,control,kombi,2007,manuell,197,150000,diesel,bmw,ja,2016-03-24 00:00:00,74535,6100
3,test,NaN,2000,manuell,150,150000,NaN,bmw,NaN,2016-04-05 00:00:00,46047,1200
4,test,limousine,1999,automatik,82,150000,benzin,mercedes_benz,nein,2016-03-09 00:00:00,25451,2500
5,control,kleinwagen,1999,manuell,60,150000,benzin,volkswagen,nein,2016-03-10 00:00:00,17379,1680


In [38]:
test_data.head()

,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,kilometer,fuelType,brand,notRepairedDamage,dateCreated,postalCode
0,test,kleinwagen,2012,manuell,68,70000,benzin,toyota,nein,2016-04-01 00:00:00,70567
1,control,limousine,1997,manuell,265,150000,benzin,audi,nein,2016-04-03 00:00:00,95448
2,test,coupe,1998,manuell,170,150000,benzin,bmw,nein,2016-03-14 00:00:00,85467
3,control,limousine,2001,automatik,102,150000,benzin,mercedes_benz,nein,2016-03-21 00:00:00,53757
4,control,limousine,1983,automatik,252,150000,benzin,bmw,nein,2016-03-30 00:00:00,47574


In [39]:
train_data.isnull().sum()

abtest                    0
vehicleType           22821
yearOfRegistration        0
gearbox               12488
powerPS                   0
kilometer                 0
fuelType              21410
brand                     0
notRepairedDamage     50238
dateCreated               0
postalCode                0
price                     0
dtype: int64

In [40]:
test_data.isnull().sum()

abtest                    0
vehicleType            7576
yearOfRegistration        0
gearbox                4041
powerPS                   0
kilometer                 0
fuelType               6570
brand                     0
notRepairedDamage     14362
dateCreated               0
postalCode                0
dtype: int64

## Now we have to handle the NaN in columns 
## 1) vehicleType
## 2) gearbox
## 3) fuelType
## 4) notRepairedDamage

## Let's handle the NaN in vehicleType

In [41]:
len(train_data[(train_data.vehicleType == 'kleinwagen') & (train_data.gearbox == 'manuell')])

54719

In [42]:
#We can see the majority of vehicles with gearbox = manuell are of vehicleType as kleinwagen, 
#So, will fill all nan in vehicleType with kleinwagen, where gearbox is manuell

In [43]:
train_data.loc[(train_data.gearbox == 'manuell') & (train_data.vehicleType.isnull()), 'vehicleType'] = 'kleinwagen'

In [44]:
test_data.loc[(test_data.gearbox == 'manuell') & (test_data.vehicleType.isnull()), 'vehicleType'] = 'kleinwagen'

In [45]:
train_data.isnull().sum()

abtest                    0
vehicleType            7929
yearOfRegistration        0
gearbox               12488
powerPS                   0
kilometer                 0
fuelType              21410
brand                     0
notRepairedDamage     50238
dateCreated               0
postalCode                0
price                     0
dtype: int64

In [46]:
train_data.loc[(train_data.gearbox == 'automatik') & (train_data.vehicleType.isnull()), 'vehicleType'] = 'limousine'

In [47]:
test_data.loc[(test_data.gearbox == 'automatik') & (test_data.vehicleType.isnull()), 'vehicleType'] = 'limousine'

In [48]:
train_data.vehicleType.isnull().sum()

5245

In [49]:
train_data.vehicleType.fillna('kleinwagen', inplace = True)

In [50]:
test_data.vehicleType.fillna('kleinwagen', inplace = True)

In [51]:
train_data.vehicleType.isnull().sum()

0

In [52]:
train_data.vehicleType.isnull().sum()

0

## Now, let's handle the NaN in gearbox

In [53]:
train_data.loc[(train_data.vehicleType == 'kleinwagen') & (train_data.gearbox.isnull()), 'gearbox'] = 'manuell'
test_data.loc[(test_data.vehicleType == 'kleinwagen') & (test_data.gearbox.isnull()), 'gearbox'] = 'manuell'

In [54]:
print(train_data.gearbox.isnull().sum())
print(test_data.gearbox.isnull().sum())

4850
1406


In [55]:
train_data.loc[(train_data.vehicleType == 'limousine') & (train_data.gearbox.isnull()), 'gearbox'] = 'automatik'
test_data.loc[(test_data.vehicleType == 'limousine') & (test_data.gearbox.isnull()), 'gearbox'] = 'automatik'

In [56]:
print(train_data.gearbox.isnull().sum())
print(test_data.gearbox.isnull().sum())

3141
873


In [57]:
train_data.gearbox.fillna('manuell', inplace = True)
test_data.gearbox.fillna('manuell', inplace = True)

In [58]:
print(train_data.gearbox.isnull().sum())
print(test_data.gearbox.isnull().sum())

0
0


## Now, let's handle NaN in fuelType

In [59]:
len(train_data[train_data.fuelType == 'andere'])

107

In [60]:
len(train_data[train_data.fuelType == 'benzin'])

170857

In [61]:
len(train_data[train_data.fuelType == 'cng'])

447

In [62]:
len(train_data[train_data.fuelType == 'diesel'])

84087

In [63]:
len(train_data[train_data.fuelType == 'elektro'])

80

In [64]:
len(train_data[train_data.fuelType == 'hybrid'])

219

In [65]:
len(train_data[train_data.fuelType == 'lpg'])

4180

In [66]:
train_data.fuelType.fillna('benzin', inplace = True)
test_data.fuelType.fillna('benzin', inplace = True)

In [67]:
print(train_data.fuelType.isnull().sum())
print(test_data.fuelType.isnull().sum())

0
0


## Now, lets handle NaN in notRepairedDamage

In [68]:
set(train_data.notRepairedDamage)

{'ja', nan, 'nein'}

In [69]:
len(train_data[train_data.notRepairedDamage == 'ja'])

26515

In [70]:
len(train_data[train_data.notRepairedDamage == 'nein'])

204634

In [71]:
train_data.notRepairedDamage.fillna('nein', inplace = True)
test_data.notRepairedDamage.fillna('nein', inplace = True)

In [72]:
print(train_data.notRepairedDamage.isnull().sum())
print(test_data.notRepairedDamage.isnull().sum())

0
0


## Now let's see if there is still any NaN or not in train and test data

In [73]:
print(train_data.isnull().sum())
print(test_data.isnull().sum())

abtest                0
vehicleType           0
yearOfRegistration    0
gearbox               0
powerPS               0
kilometer             0
fuelType              0
brand                 0
notRepairedDamage     0
dateCreated           0
postalCode            0
price                 0
dtype: int64
abtest                0
vehicleType           0
yearOfRegistration    0
gearbox               0
powerPS               0
kilometer             0
fuelType              0
brand                 0
notRepairedDamage     0
dateCreated           0
postalCode            0
dtype: int64


## Great, no NaN, now we can go ahead with the data

In [74]:
print(train_data.shape)
print(test_data.shape)

(281387, 12)
(74343, 11)


In [75]:
train_data.head()

,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,kilometer,fuelType,brand,notRepairedDamage,dateCreated,postalCode,price
0,control,kombi,2009,manuell,105,150000,diesel,volkswagen,nein,2016-03-06 00:00:00,24220,5999
2,control,kombi,2007,manuell,197,150000,diesel,bmw,ja,2016-03-24 00:00:00,74535,6100
3,test,kleinwagen,2000,manuell,150,150000,benzin,bmw,nein,2016-04-05 00:00:00,46047,1200
4,test,limousine,1999,automatik,82,150000,benzin,mercedes_benz,nein,2016-03-09 00:00:00,25451,2500
5,control,kleinwagen,1999,manuell,60,150000,benzin,volkswagen,nein,2016-03-10 00:00:00,17379,1680


In [76]:
abtest_list = list(set(train_data.abtest))

In [77]:
vehicleType_list = list(set(train_data.vehicleType))

In [78]:
gearbox_list = list(set(train_data.gearbox))

In [79]:
fuelType_list = list(set(train_data.fuelType))

In [80]:
brand_list = list(set(train_data.brand))

In [81]:
notRepairedDamage_list = list(set(train_data.notRepairedDamage))

In [82]:
from datetime import datetime
d_year = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).year

train_data['d_create_year'] = train_data['dateCreated'].map(d_year)
train_data['age'] = train_data['d_create_year'] - train_data['yearOfRegistration']
del train_data['dateCreated']
del train_data['d_create_year']


test_data['d_create_year'] = test_data['dateCreated'].map(d_year)
test_data['age'] = test_data['d_create_year'] - test_data['yearOfRegistration']
del test_data['dateCreated']
del test_data['d_create_year']


In [83]:
del train_data['yearOfRegistration']
del test_data['yearOfRegistration']

In [84]:
train_data.head()

,abtest,vehicleType,gearbox,powerPS,kilometer,fuelType,brand,notRepairedDamage,postalCode,price,age
0,control,kombi,manuell,105,150000,diesel,volkswagen,nein,24220,5999,7
2,control,kombi,manuell,197,150000,diesel,bmw,ja,74535,6100,9
3,test,kleinwagen,manuell,150,150000,benzin,bmw,nein,46047,1200,16
4,test,limousine,automatik,82,150000,benzin,mercedes_benz,nein,25451,2500,17
5,control,kleinwagen,manuell,60,150000,benzin,volkswagen,nein,17379,1680,17


In [85]:
def get_integer1(s):
    return abtest_list.index(s)

In [86]:
def get_integer2(s):
    return vehicleType_list.index(s)

In [87]:
def get_integer3(s):
    return gearbox_list.index(s)

In [88]:
def get_integer4(s):
    return fuelType_list.index(s)

In [89]:
def get_integer5(s):
    return brand_list.index(s)

In [90]:
def get_integer6(s):
    return notRepairedDamage_list.index(s)

In [91]:
train_data['abtest_int'] = train_data['abtest'].apply(get_integer1)
test_data['abtest_int'] = test_data['abtest'].apply(get_integer1)
del train_data['abtest']
del test_data['abtest']

In [92]:
train_data['vehicleType_int'] = train_data['vehicleType'].apply(get_integer2)
test_data['vehicleType_int'] = test_data['vehicleType'].apply(get_integer2)
del train_data['vehicleType']
del test_data['vehicleType']

In [93]:
train_data['gearbox_int'] = train_data['gearbox'].apply(get_integer3)
test_data['gearbox_int'] = test_data['gearbox'].apply(get_integer3)
del train_data['gearbox']
del test_data['gearbox']

In [94]:
train_data['fuelType_int'] = train_data['fuelType'].apply(get_integer4)
test_data['fuelType_int'] = test_data['fuelType'].apply(get_integer4)
del train_data['fuelType']
del test_data['fuelType']

In [95]:
train_data['brand_int'] = train_data['brand'].apply(get_integer5)
test_data['brand_int'] = test_data['brand'].apply(get_integer5)
del train_data['brand']
del test_data['brand']

In [96]:
train_data['notRepairedDamage_int'] = train_data['notRepairedDamage'].apply(get_integer6)
test_data['notRepairedDamage_int'] = test_data['notRepairedDamage'].apply(get_integer6)
del train_data['notRepairedDamage']
del test_data['notRepairedDamage']

In [97]:
train_data.head()

,powerPS,kilometer,postalCode,price,age,abtest_int,vehicleType_int,gearbox_int,fuelType_int,brand_int,notRepairedDamage_int
0,105,150000,24220,5999,7,1,3,1,0,23,0
2,197,150000,74535,6100,9,1,3,1,0,0,1
3,150,150000,46047,1200,16,0,2,1,3,0,0
4,82,150000,25451,2500,17,0,1,0,3,10,0
5,60,150000,17379,1680,17,1,2,1,3,23,0


In [98]:
test_data.head()

,powerPS,kilometer,postalCode,age,abtest_int,vehicleType_int,gearbox_int,fuelType_int,brand_int,notRepairedDamage_int
0,68,70000,70567,4,0,2,1,3,12,0
1,265,150000,95448,19,1,1,1,3,5,0
2,170,150000,85467,18,0,0,1,3,0,0
3,102,150000,53757,15,1,1,0,3,10,0
4,252,150000,47574,33,1,1,0,3,0,0


In [99]:
y_train = train_data['price'].values

In [100]:
del train_data['price']

In [101]:
x_train = train_data.values

In [102]:
x_test = test_data.values

In [103]:
x_train.shape

(281387, 10)

In [104]:
x_test.shape

(74343, 10)

In [105]:
y_train.shape

(281387,)

In [106]:
x_train1, x_test1, y_train1, y_test1 = train_test_split(x_train, y_train, random_state = 0)

In [107]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train1 = scaler.fit_transform(x_train1)
x_test1 = scaler.transform(x_test1)

C:\Users\NKS\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [108]:
rfr = RandomForestRegressor()
rfr.fit(x_train1, y_train1)
rfr.score(x_test1, y_test1)

0.7975710519061965

In [109]:
y_pred_submit = rfr.predict(x_test)
np.savetxt("predictions_used_car.csv", y_pred_submit, '%.5f', delimiter = ',')